## Sending data to Kafka server

This notebook uses the [Python client for the Apache Kafka distributed stream processing system](http://kafka-python.readthedocs.io/en/master/index.html) to send messages to a Kafka cluster. 

In this example, the messages are data generated from a linear model with $n$ input variables, i.e., 

$$
y =x^T \beta +w
$$
with $x, \beta \in \mathbb{R}^n$, and $y, w \in \mathbb{R}$. $w$ is Gaussian noise.

Messages are sent every $interval$ seconds. They are list of size (n+2) where:
* First element is the counter 
* Second element is $y$ 
* Third to last elements are $x$ values (size $n$)


In [ ]:
from kafka import KafkaProducer
import time
import numpy as np

In [ ]:
producer = KafkaProducer(bootstrap_servers='kafka1:19092,kafka2:29092,kafka3:39092')

In [ ]:
i=0 #Initialise counter

n=10   # number of inputs

beta=np.zeros(n) ## first and last parameters are 1, others are zeros
beta[1]=1   
beta[-1]=1
beta.shape=(n,1)


#Loop for sending messages to Kafka with the topic dataLinearModel

while True:
    x=np.random.rand(1,n)[0]
    y=float(x.dot(beta))+0.1*np.random.rand(1)[0] ## y =x^T beta +w
    message=np.array2string(np.append([i,y],x),separator=",") 
    print(message)
    producer.send('dataLinearModel', message.encode())
    i=i+1
    time.sleep(1)